This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/xgboost-upload-tutorials).

## XGBoost RF Regressor Example

The following tutorial demonstrates deploying and serving an XGBoost RF Regressor model to Wallaroo.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Arbitrary Python models, also known as Bring Your Own Predict (BYOP)](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

## Goal

Upload, deploy, and serve a sample XGBoost RF Regressor model.

### Resources

This tutorial provides the following:

* Models:
  * `./models/xgb_rf_regressor.pkl`: The sample XGBoost model that receives the `sklearn.datasets.load_diabetes` dataset.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.
* The notebook "Wallaroo Run Anywhere Model Drift Observability with Assays: Preparation" has been run, and the model edge deployments executed.

## Steps

### Import Libraries

The first step is to import the libraries we will need.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_diabetes
from xgboost import train, DMatrix
from sklearn.model_selection import train_test_split

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [3]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

workspace_name = f'xgb-rf-regressor-conversion'
pipeline_name = f'xgb-rf-regressor-conversion'

model_name = 'xgb-rf-regressor-conversion'
model_file_name = './models/xgb_rf_regressor.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name, wl)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload XGBoost Model

XGBoost models are uploaded to Wallaroo through the `wallaroo.client.Client.upload_model` method.

#### Upload XGBoost Model Parameters

The following parameters are required for XGBoost models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a XGBoost model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as the `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload XGBoost Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |#

### Configure Input and Output Schemas

First we configure the input and output schemas in PyArrow format.

In [5]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=10))
])

output_schema = pa.schema([
    pa.field('predictions', pa.float32()),
])

### Upload Model Example

With the input and output schemas defined, we now upload the XGBoost model.

In [6]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime............incompatible

Model is pending loading to a container runtime..
Model is attempting loading to a container runtime..........successful

Ready


Name,xgb-rf-regressor-conversion
Version,391ec81f-2210-464a-b0b1-89c731fa0683
File Name,xgb_rf_regressor.pkl
SHA,ead81583402e6b17f14402b848c6ae6eb7854578f607ea9baf63379a6f60539c
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4806
Architecture,None
Acceleration,None
Updated At,2024-28-Mar 15:01:19


### Deploy Pipeline

With the model uploaded and packaged, we add the model as a pipeline model step.  For our deployment, we will set a minimum deployment configuration - this is the amount of resources the deployed pipeline uses from the cluster.

Once set, we deploy the pipeline, which allocates the assigned resources for the cluster and makes it available for inference requests.

In [7]:
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [8]:
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.0.185',
   'name': 'engine-85cfd5c66b-mcpzs',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'xgb-rf-regressor-conversion',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 68,
       'input_schema': '/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAEAAAAyP///wAAARAUAAAAJAAAAAQAAAABAAAAMAAAAAYAAABpbnB1dHMAAAAABgAIAAQABgAAAAoAAAAQABQACAAGAAcADAAAABAAEAAAAAAAAQMQAAAAHAAAAAQAAAAAAAAABAAAAGl0ZW0AAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'model_version_id': 37,
       'output_schema': '/////4AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAUAAAAEAAUAAgABgAHAAwAAAAQABAAAAAAAAEDEAAAACQAAAAEAAAAAAAAAAsAAABwcmVkaWN0aW9ucwAAAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'runtime': 'flight',
       'sidekick_uri': None,
       'tensor_fields': None}

### Run Sample Inference

The dataset is from the `sklearn.datasets.load_breast_cancer` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [13]:
data = load_diabetes(as_frame=True)

X = data['data'].values[:100]
y = data['target'][:100]

In [14]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": X[:100].tolist()})

In [15]:
results = pipeline.infer(dataframe)
results

,time,in.inputs,out.predictions,anomaly.count
0,2024-03-28 15:03:13.736,"[0.0380759064, 0.0506801187, 0.0616962065, 0.0...",222.492130,0
1,2024-03-28 15:03:13.736,"[-0.0018820165, -0.0446416365, -0.0514740612, ...",85.170210,0
2,2024-03-28 15:03:13.736,"[0.0852989063, 0.0506801187, 0.0444512133, -0....",156.588300,0
3,2024-03-28 15:03:13.736,"[-0.0890629394, -0.0446416365, -0.0115950145, ...",180.512100,0
4,2024-03-28 15:03:13.736,"[0.0053830604, -0.0446416365, -0.0363846922, 0...",105.263480,0
...,...,...,...,...
95,2024-03-28 15:03:13.736,"[-0.0709002471, -0.0446416365, -0.0579409337, ...",139.580600,0
96,2024-03-28 15:03:13.736,"[0.0562385987, 0.0506801187, 0.009961227, 0.04...",207.162200,0
97,2024-03-28 15:03:13.736,"[-0.0273097857, -0.0446416365, 0.0886415084, -...",166.432820,0
98,2024-03-28 15:03:13.736,"[0.0017505219, 0.0506801187, -0.0051281421, -0...",124.094310,0


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the cluster.

In [16]:
pipeline.undeploy()

name,xgb-rf-regressor-conversion
created,2024-03-28 14:59:03.946327+00:00
last_updated,2024-03-28 15:01:20.866942+00:00
deployed,False
arch,None
accel,None
tags,
versions,"a0addd85-4715-47c8-bf34-a48af375ca2c, a7a0132b-b13e-4fd6-87af-5eac29b3ceac"
steps,xgb-rf-regressor-conversion
published,False
